# loop until you can reserve an A100 GPU
# API Ref: https://cloud.google.com/python/docs/reference/compute/latest

In [ ]:
!pip install --upgrade -q google-cloud-compute

In [ ]:
from google.cloud import compute_v1
from google.api_core import exceptions as core_exceptions
import datetime
import logging
import uuid
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

In [ ]:
project = "CHANGE_IT"
zone = "us-central1-c"
machines = 1
machine_type = "a2-highgpu-1g"
retries = 100
reservation_name = "gpu-reservation-{}".format(uuid.uuid4())

In [ ]:
client = compute_v1.ReservationsClient()

request = { #https://cloud.google.com/python/docs/reference/compute/latest/google.cloud.compute_v1.types.InsertReservationRequest
    "project": project,
    "zone": zone,
    "reservation_resource" : { #https://cloud.google.com/python/docs/reference/compute/latest/google.cloud.compute_v1.types.Reservation
        "name": reservation_name,
        "share_settings": { #https://cloud.google.com/python/docs/reference/compute/latest/google.cloud.compute_v1.types.ShareSettings
            "share_type": "LOCAL", #https://cloud.google.com/python/docs/reference/compute/latest/google.cloud.compute_v1.types.ShareSettings#google_cloud_compute_v1_types_ShareSettings_ShareType
        },
    "specific_reservation": {
        "count": machines,
        "instance_properties": { #https://cloud.google.com/python/docs/reference/compute/latest/google.cloud.compute_v1.types.AllocationSpecificSKUAllocationReservedInstanceProperties
            
            "guest_accelerators" : [{  #https://cloud.google.com/python/docs/reference/compute/latest/google.cloud.compute_v1.types.AcceleratorConfig
                "accelerator_count": 1,
                "accelerator_type": "nvidia-tesla-a100"
            }],
            "machine_type": machine_type
        }
    }
    }   
}
logging.info("{}: Requesting resource".format(datetime.datetime.now()))
for r in range(1,retries):
    try: 
        operation = client.insert(request=request)
        response = operation.result()
        if not response: 
            break
    except core_exceptions.ServiceUnavailable:
        logging.info("Resource unavailable, retrying...{}".format(r+1))
logging.info("{}: done".format(datetime.datetime.now()))

